# Modelagem de classificação de clientes com dados da saúde suplementar

Vamos usar os [dados obtidos anteriormente](https://github.com/VFLins/hospitalar_data_ans/blob/main/notebooks/collect_routine_ans.ipynb) e o conhecimento que obtivemos na sua [análise exploratória](https://vflins.github.io/hospitalar_data_ans/pages/eda.html) para produzir uma solução de dados com valor de negócio para empresas do **setor de operação de saúde.**

A proposta deste modelo é produzir um indicador do *"custo potencial de aquisição"* de um cliente. Esta solução pode facilitar grandemente a precificação dos produtos de saúde, além disso, neste processo podemos acabar aprendendo mais sobre os diferentes tipos de perfis de clientes e suas necessidades!

In [1]:
# imports
import logging
loghandler = logging.FileHandler(".log")
logging.getLogger().addHandler(loghandler)


from pyspark.sql import SparkSession
from getpass import getpass
from pyspark.sql.types import StructType, StructField, TimestampNTZType, DateType, FloatType, StringType
from pyspark.sql.functions import monotonically_increasing_id

In [3]:
# spark session

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [4]:
# variaveis de conexao

drivername = "oracle.jdbc.OracleDriver"
WALLET_PLACE = "/home/user/Downloads/wallet_demodb"
URL = f"jdbc:oracle:thin:@demodb_medium?TNS_ADMIN={WALLET_PLACE}"
USR = "ADMIN"
PWD = getpass("Insira a senha de administrador do banco de dados: ")

Insira a senha de administrador do banco de dados:  ········


In [5]:
# conjuntos de dados

tablename = "HOSP_CONS"
query = f"SELECT * FROM {tablename} ORDER BY dbms_random.value FETCH FIRST 624118 ROWS ONLY"

cons_df = spark.read.format("jdbc")\
    .option("driver", drivername)\
    .option("query", query)\
    .option("url", URL)\
    .option("user", USR)\
    .option("password", PWD)\
    .load()\
    .toPandas()


tablename = "HOSP_DET"
query = f"SELECT * FROM {tablename} ORDER BY dbms_random.value FETCH FIRST 4016936 ROWS ONLY"

det_df = spark.read.format("jdbc")\
    .option("driver", drivername)\
    .option("query", query)\
    .option("url", URL)\
    .option("user", USR)\
    .option("password", PWD)\
    .load()\
    .toPandas()

24/04/21 11:24:45 ERROR Executor: Exception in task 0.0 in stage 1.0 (TID 1)/ 1]
java.sql.SQLException: ORA-01652: unable to extend temp segment by 128 in tablespace TEMP

https://docs.oracle.com/error-help/db/ora-01652/
	at oracle.jdbc.driver.T4CTTIoer11.processError(T4CTTIoer11.java:702)
	at oracle.jdbc.driver.T4CTTIoer11.processError(T4CTTIoer11.java:608)
	at oracle.jdbc.driver.T4C8Oall.processError(T4C8Oall.java:1330)
	at oracle.jdbc.driver.T4CTTIfun.receive(T4CTTIfun.java:1102)
	at oracle.jdbc.driver.T4CTTIfun.doRPC(T4CTTIfun.java:456)
	at oracle.jdbc.driver.T4C8Oall.doOALL(T4C8Oall.java:498)
	at oracle.jdbc.driver.T4CPreparedStatement.doOall8(T4CPreparedStatement.java:272)
	at oracle.jdbc.driver.T4CPreparedStatement.executeForDescribe(T4CPreparedStatement.java:1198)
	at oracle.jdbc.driver.OracleStatement.prepareDefineBufferAndExecute(OracleStatement.java:1390)
	at oracle.jdbc.driver.OracleStatement.executeMaybeDescribe(OracleStatement.java:1269)
	at oracle.jdbc.driver.OracleState

Py4JJavaError: An error occurred while calling o45.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (172.18.59.98 executor driver): java.sql.SQLException: ORA-01652: unable to extend temp segment by 128 in tablespace TEMP

https://docs.oracle.com/error-help/db/ora-01652/
	at oracle.jdbc.driver.T4CTTIoer11.processError(T4CTTIoer11.java:702)
	at oracle.jdbc.driver.T4CTTIoer11.processError(T4CTTIoer11.java:608)
	at oracle.jdbc.driver.T4C8Oall.processError(T4C8Oall.java:1330)
	at oracle.jdbc.driver.T4CTTIfun.receive(T4CTTIfun.java:1102)
	at oracle.jdbc.driver.T4CTTIfun.doRPC(T4CTTIfun.java:456)
	at oracle.jdbc.driver.T4C8Oall.doOALL(T4C8Oall.java:498)
	at oracle.jdbc.driver.T4CPreparedStatement.doOall8(T4CPreparedStatement.java:272)
	at oracle.jdbc.driver.T4CPreparedStatement.executeForDescribe(T4CPreparedStatement.java:1198)
	at oracle.jdbc.driver.OracleStatemen

# 1 Criando um *target*

Vamos abordar este problema usando um modelo de classificação. Para isto vamos precisar de um variável target binária que separe os clientes entre "alto custo" e "baixo custo". Usaremos a probabilidade estimada de um cliente pertencer a um grupo ou à outro como o *score* que indica seu custo potencial.

In [4]:
print(f"Consolidada:\n{cons_df.schema.names}", f"Detalhada:\n{det_df.schema.names}", sep="\n\n")

Consolidada:
['ID_EVENTO_ATENCAO_SAUDE', 'ID_PLANO', 'FAIXA_ETARIA', 'SEXO', 'CD_MUNICIPIO_BENEFICIARIO', 'PORTE', 'CD_MODALIDADE', 'NM_MODALIDADE', 'CD_MUNICIPIO_PRESTADOR', 'UF_PRESTADOR', 'TEMPO_DE_PERMANENCIA', 'ANO_MES_EVENTO', 'CD_CARATER_ATENDIMENTO', 'CD_TIPO_INTERNACAO', 'CD_REGIME_INTERNACAO', 'CD_MOTIVO_SAIDA', 'CID_1', 'CID_2', 'CID_3', 'CID_4', 'QT_DIARIA_ACOMPANHANTE', 'QT_DIARIA_UTI', 'IND_ACIDENTE_DOENCA', 'LG_VALOR_PREESTABELECIDO']

Detalhada:
['ID_EVENTO_ATENCAO_SAUDE', 'UF_PRESTADOR', 'TEMPO_DE_PERMANENCIA', 'ANO_MES_EVENTO', 'CD_PROCEDIMENTO', 'CD_TABELA_REFERENCIA', 'QT_ITEM_EVENTO_INFORMADO', 'VL_ITEM_EVENTO_INFORMADO', 'VL_ITEM_PAGO_FORNECEDOR', 'IND_PACOTE', 'IND_TABELA_PROPRIA']


## 1.1 A especificidade de cada conjunto de dados

Para obter o custo de um cliente, seria ideal ter um conjunto de dados que representa um cliente único em cada linha, em vez disto, temos:

- A nossa base de dados **consolidada** apresentando um atendimento médico para cada linha
- A base de dados **detalhada** apresentando um procedimento do atendimento médico para cada linha

Podemos perceber que as variáveis que descrevem o cliente (exemplo: `"FAIXA_ETARIA"`, `"SEXO"`, etc.) estão presentes na tabela *consolidada*. Para obter um conjunto no formato que precisamos para modelagem, será necessário agregar algumas características dos consumidores presentes na tabela consolidada.

Depois que tivermos como "sinalizar" quais são os clientes únicos presentes na tabela **consolidada**, podemos extrair mais informações sobre seus respctivos eventos hospitalares na tabela **detalhada**.

In [3]:
from zipfile import ZipFile
from io import BytesIO
from os import makedirs
import requests


response = requests.get("https://archive.ics.uci.edu/static/public/34/diabetes.zip")
datafile = "diabetes-data.tar.Z"
target_path = f"/home/user/code/hospitalar_data_ans/data/"
makedirs(target_path, exist_ok=True)

with ZipFile(BytesIO(response.content)) as zipf:
    if datafile in zipf.namelist():
        with zipf.open(datafile) as origin:
            with open(f"{target_path}/data.txt", "wb") as target:
                target.write(origin.read())

In [35]:
df = df.withColumn("id", monotonically_increasing_id() + 1)